In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address      Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.0.4  70.26 KiB  16      47.6%             a0ba96a2-9f3b-4697-a5cd-a1109d094155  rack1
UN  192.168.0.2  70.27 KiB  16      93.0%             ac5e7d77-7daf-4a22-b35c-2819d2b1f7c9  rack1
UN  192.168.0.3  70.27 KiB  16      59.3%             a5e972ef-4fab-43eb-b991-8b7d59864bde  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("drop keyspace if exists weather")

In [4]:
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")

In [5]:
cass.execute("use weather")

In [6]:
cass.execute("""
create type weather.station_record(
    tmin INT,
    tmax INT
)
""")

In [7]:
cass.execute("""
create table weather.stations(
    id  TEXT,
    name TEXT static,
    date DATE,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

In [8]:
print(cass.execute("describe table weather.stations").one().create_statement)

CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [9]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-046a3db5-6c66-4b0d-892a-10fdb99296fa;1.0
	confs: [default]


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central


	found com.typesafe#config;1.4.1 in central


...

	[SUCCESSFUL ] org.scala-lang#scala-reflect;2.12.11!scala-reflect.jar (148ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/native-protocol/1.5.0/native-protocol-1.5.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#native-protocol;1.5.0!native-protocol.jar(bundle) (38ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-shaded-guava/25.1-jre-graal-sub-1/java-driver-shaded-guava-25.1-jre-graal-sub-1.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1!java-driver-shaded-guava.jar (97ms)
downloading https://repo1.maven.org/maven2/com/typesafe/config/1.4.1/config-1.4.1.jar ...
	[SUCCESSFUL ] com.typesafe#config;1.4.1!config.jar(bundle) (31ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.26/slf4j-api-1.7.26.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.26!slf4j-api.jar (30ms)


downloading https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/4.1.18/metrics-core-4.1.18.jar ...
	[SUCCESSFUL ] io.dropwizard.metrics#metrics-core;4.1.18!metrics-core.jar(bundle) (31ms)
downloading https://repo1.maven.org/maven2/org/hdrhistogram/HdrHistogram/2.1.12/HdrHistogram-2.1.12.jar ...
	[SUCCESSFUL ] org.hdrhistogram#HdrHistogram;2.1.12!HdrHistogram.jar(bundle) (32ms)
downloading https://repo1.maven.org/maven2/org/reactivestreams/reactive-streams/1.0.3/reactive-streams-1.0.3.jar ...
	[SUCCESSFUL ] org.reactivestreams#reactive-streams;1.0.3!reactive-streams.jar (29ms)
downloading https://repo1.maven.org/maven2/com/github/stephenc/jcip/jcip-annotations/1.0-1/jcip-annotations-1.0-1.jar ...
	[SUCCESSFUL ] com.github.stephenc.jcip#jcip-annotations;1.0-1!jcip-annotations.jar (27ms)
downloading https://repo1.maven.org/maven2/com/github/spotbugs/spotbugs-annotations/3.1.12/spotbugs-annotations-3.1.12.jar ...
	[SUCCESSFUL ] com.github.spotbugs#spotbugs-annotations;3.1.12!

	[SUCCESSFUL ] com.datastax.oss#java-driver-query-builder;4.13.0!java-driver-query-builder.jar(bundle) (35ms)
:: resolution report :: resolve 5488ms :: artifacts dl 1387ms
	:: modules in use:
	com.datastax.oss#java-driver-core-shaded;4.13.0 from central in [default]
	com.datastax.oss#java-driver-mapper-runtime;4.13.0 from central in [default]
	com.datastax.oss#java-driver-query-builder;4.13.0 from central in [default]
	com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 from central in [default]
	com.datastax.oss#native-protocol;1.5.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 from central in [default]
	com.github.spotbugs#spotbugs-annotations;3.1.12 from central in [default]
	com.github.stephenc.jcip#jcip-annotations;1.0-1 from central in [default]
	com.google.code.findbugs#jsr305;3.0.2 from central in [default]
	com.thoughtworks.paranamer#parana

23/11/20 23:49:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
import pandas as pd

In [11]:
from pyspark.sql.functions import col, expr
df = spark.read.text("ghcnd-stations.txt")

In [12]:
df.toPandas()

,value
0,ACW00011604 17.1167 -61.7833 10.1 ST JO...
1,ACW00011647 17.1333 -61.7833 19.2 ST JO...
2,AE000041196 25.3330 55.5170 34.0 SHARJ...
3,AEM00041194 25.2550 55.3640 10.4 DUBAI...
4,AEM00041217 24.4330 54.6510 26.8 ABU D...
...,...
123341,ZI000067969 -21.0500 29.3670 861.0 WEST ...
123342,ZI000067975 -20.0670 30.8670 1095.0 MASVI...
123343,ZI000067977 -21.0170 31.5830 430.0 BUFFA...
123344,ZI000067983 -20.2000 32.6160 1132.0 CHIPI...


In [13]:
from pyspark.sql.functions import col, expr
df = df.select(
    expr("substring(value, 0, 11)").alias("id"),
    expr("substring(value, 39, 2)").alias("state"),
    expr("substring(value, 42, 30)").alias("name"),
)
df.toPandas()

,id,state,name
0,ACW00011604,,ST JOHNS COOLIDGE FLD
1,ACW00011647,,ST JOHNS
2,AE000041196,,SHARJAH INTER. AIRP
3,AEM00041194,,DUBAI INTL
4,AEM00041217,,ABU DHABI INTL
...,...,...,...
123341,ZI000067969,,WEST NICHOLSON
123342,ZI000067975,,MASVINGO
123343,ZI000067977,,BUFFALO RANGE
123344,ZI000067983,,CHIPINGE


In [14]:
df = df.where(df.state == 'WI')
df

DataFrame[id: string, state: string, name: string]

In [15]:
data = df.collect()

In [16]:
insert_df = cass.prepare("""
INSERT INTO weather.stations (id, name)
VALUES (?, ?)
""")

for row in data:
    cass.execute(insert_df, (row.id, row.name))

In [17]:
#q1
count_stations = cass.execute("""
SELECT COUNT(*)
FROM weather.stations
""")
count_stations.one()[0]

1313

In [18]:
#q2
stations = cass.execute("""
SELECT name
FROM weather.stations
WHERE id = 'USW00014837'
""")
stations.one()[0]

'MADISON DANE CO RGNL AP       '

In [19]:
#q3
stations = cass.execute("""
SELECT token(id)
FROM weather.stations
WHERE id = 'USC00470273'
""")
stations.one()[0]

-9014250178872933741

In [20]:
#q4 @@@@@@@@@@@@@@@@@@@Handle the case where the ring "wraps around" 
stations = cass.execute("""
SELECT token(id)
FROM weather.stations
WHERE token(id) > token('USC00470273')
""")
stations.one()[0]

-9008888552508841246